In [3]:
import joblib, math, os
import torch
import pandas as pd
import numpy as np
from scipy.constants import physical_constants, degree
from tqdm import tqdm
os.chdir('/local/disk4/chrenx/fog-challenge')
print(os.getcwd())

def sample_normalize(sample):
    """Mean-std normalization function. 

    Args:
        sample: (N,)
    Returns:
        normalized_sample: (N,)
    """
    if not isinstance(sample, torch.Tensor):
        sample = torch.tensor(sample)
    mean = torch.mean(sample)
    std = torch.std(sample)
    # Normalize the sample and handle division by zero
    eps = 1e-8
    normalized_sample = (sample - mean) / (std + eps)
    return normalized_sample  # (N,)

/local/disk4/chrenx/fog-challenge


In [8]:
a = np.array(['wer'] * 3)
a

array(['wer', 'wer', 'wer'], dtype='<U3')

In [4]:
dpath = ['data/rectified_data/all_data/all_daphnet.p', 
         'data/rectified_data/all_data/all_kaggle_pd_data_defog.p',
         'data/rectified_data/all_data/all_kaggle_pd_data_tdcsfog.p',
         'data/rectified_data/all_data/all_turn_in_place.p']
for p in dpath:
    data = joblib.load(p)
    unique_ori_filename = set()
    for key,value in data.items():
        unique_ori_filename.add(value['ori_filename'])
    print(f"{p.split('/')[-1]}: {len(unique_ori_filename)}, {len(data.keys())}")

all_daphnet.p: 17, 35
all_kaggle_pd_data_defog.p: 91, 1027
all_kaggle_pd_data_tdcsfog.p: 833, 833
all_turn_in_place.p: 71, 71


# Submission

In [20]:
daphnet_all_dpath = 'data/rectified_data/all_data/all_daphnet.p'
daphnet_test_dpath = 'data/rectified_data/all_data/test_daphnet_window1024_randomaug.p' 
all_data = joblib.load(daphnet_all_dpath)
test_data = joblib.load(daphnet_test_dpath)

test_ori_filename = set()

for key, value in test_data.items():
    test_ori_filename.add(value['ori_filename'])
print(len(test_ori_filename), test_ori_filename)


def dict_to_csv(dictionary, csv_filename):
    # Convert each tensor in the dictionary to a list
    converted_dict = {key: value.tolist() for key, value in dictionary.items()}
    
    # Create a DataFrame from the dictionary
    df = pd.DataFrame(converted_dict)
    
    # Write the DataFrame to a CSV file
    df.to_csv(csv_filename, index=False)

for key, value in all_data.items():
    if value['ori_filename'] not in test_ori_filename:
        continue
    input_dict = {
        'L_Ankle_Acc_X': value['L_Ankle_Acc_AP'],
        'L_Ankle_Acc_Y': value['L_Ankle_Acc_V'],
        'L_Ankle_Acc_Z': value['L_Ankle_Acc_ML'],
        'L_MidLatThigh_Acc_X': value['L_MidLatThigh_Acc_AP'],
        'L_MidLatThigh_Acc_Y': value['L_MidLatThigh_Acc_V'],
        'L_MidLatThigh_Acc_Z': value['L_MidLatThigh_Acc_ML'],
        'LowerBack_Acc_X': value['LowerBack_Acc_AP'],
        'LowerBack_Acc_Y': value['LowerBack_Acc_V'],
        'LowerBack_Acc_Z': value['LowerBack_Acc_ML'],
    }
    csv_filename = f'data/submission_test_data/{key}.csv'
    dict_to_csv(input_dict, csv_filename)

2 {'S02R01.txt', 'S03R02.txt'}


In [ ]:
daphnet_all_dpath = 'data/rectified_data/all_data/all_daphnet.p'
daphnet_test_dpath = 'data/rectified_data/all_data/test_daphnet_window1024_randomaug.p' 
all_data = joblib.load(daphnet_all_dpath)
test_data = joblib.load(daphnet_test_dpath)

test_ori_filename = set()

for key, value in test_data.items():
    test_ori_filename.add(value['ori_filename'])
print(len(test_ori_filename), test_ori_filename)


def dict_to_csv(dictionary, csv_filename):
    # Convert each tensor in the dictionary to a list
    converted_dict = {key: value.tolist() for key, value in dictionary.items()}
    
    # Create a DataFrame from the dictionary
    df = pd.DataFrame(converted_dict)
    
    # Write the DataFrame to a CSV file
    df.to_csv(csv_filename, index=False)

for key, value in all_data.items():
    if value['ori_filename'] not in test_ori_filename:
        continue
    input_dict = {
        'L_Ankle_Acc_X': value['L_Ankle_Acc_AP'],
        'L_Ankle_Acc_Y': value['L_Ankle_Acc_V'],
        'L_Ankle_Acc_Z': value['L_Ankle_Acc_ML'],
        'L_MidLatThigh_Acc_X': value['L_MidLatThigh_Acc_AP'],
        'L_MidLatThigh_Acc_Y': value['L_MidLatThigh_Acc_V'],
        'L_MidLatThigh_Acc_Z': value['L_MidLatThigh_Acc_ML'],
        'LowerBack_Acc_X': value['LowerBack_Acc_AP'],
        'LowerBack_Acc_Y': value['LowerBack_Acc_V'],
        'LowerBack_Acc_Z': value['LowerBack_Acc_ML'],
    }
    csv_filename = f'data/submission_test_data/{key}.csv'
    dict_to_csv(input_dict, csv_filename)

# Daphnet

In [2]:
dpath = 'data/dataset_fog_release/dataset'

all_txt_files = os.listdir(dpath)

all_data = {}

physical_const = physical_constants['standard acceleration of gravity'][0]


count = 0
for txt_file in tqdm(all_txt_files, total=len(all_txt_files)):
    file = open(os.path.join(dpath, txt_file))
    
    all_lines = file.readlines()
    
    ankle_acc_ap, ankle_acc_v, ankle_acc_ml = [], [], []
    mid_thigh_acc_ap, mid_thigh_acc_v, mid_thigh_acc_ml = [], [], []
    lowerback_acc_ap, lowerback_acc_v, lowerback_acc_ml = [], [], []
    gt = []
    
    # print(txt_file)
    # print(all_lines[0])
    # break
    
    for line in all_lines:
        line = line.strip().split(' ')
        if int(line[-1]) == 0 and len(ankle_acc_ap) < 700:
            ankle_acc_ap, ankle_acc_v, ankle_acc_ml = [], [], []
            mid_thigh_acc_ap, mid_thigh_acc_v, mid_thigh_acc_ml = [], [], []
            lowerback_acc_ap, lowerback_acc_v, lowerback_acc_ml = [], [], []
            gt = []
            continue
        elif int(line[-1]) == 0 and len(ankle_acc_ap) >= 700:
            count += 1
            event = ['stand or walk or turn'] * len(ankle_acc_ap)
            all_data[f'rectified_{count}_daphnet'] = {
                'ori_filename':txt_file,
                'event': np.array(event), 
                'L_Ankle_Acc_AP': sample_normalize(torch.tensor(ankle_acc_ap, dtype=torch.float32)),
                'L_Ankle_Acc_V': sample_normalize(torch.tensor(ankle_acc_v, dtype=torch.float32)),
                'L_Ankle_Acc_ML': sample_normalize(torch.tensor(ankle_acc_ml, dtype=torch.float32)),
                'L_MidLatThigh_Acc_AP': sample_normalize(torch.tensor(mid_thigh_acc_ap, dtype=torch.float32)),
                'L_MidLatThigh_Acc_V': sample_normalize(torch.tensor(mid_thigh_acc_v, dtype=torch.float32)),
                'L_MidLatThigh_Acc_ML': sample_normalize(torch.tensor(mid_thigh_acc_ml, dtype=torch.float32)),
                'LowerBack_Acc_AP': sample_normalize(torch.tensor(lowerback_acc_ap, dtype=torch.float32)),
                'LowerBack_Acc_V': sample_normalize(torch.tensor(lowerback_acc_v, dtype=torch.float32)),
                'LowerBack_Acc_ML': sample_normalize(torch.tensor(lowerback_acc_ml, dtype=torch.float32)),
                'gt': torch.tensor(gt, dtype=torch.float32),
            }
            ankle_acc_ap, ankle_acc_v, ankle_acc_ml = [], [], []
            mid_thigh_acc_ap, mid_thigh_acc_v, mid_thigh_acc_ml = [], [], []
            lowerback_acc_ap, lowerback_acc_v, lowerback_acc_ml = [], [], []
            gt = []
        else:
            ankle_acc_ap.append(float(line[1]) * physical_const / 1000)
            ankle_acc_v.append(float(line[2]) * physical_const / 1000)
            ankle_acc_ml.append(float(line[3]) * physical_const / 1000)
            mid_thigh_acc_ap.append(float(line[4]) * physical_const / 1000)
            mid_thigh_acc_v.append(float(line[5]) * physical_const / 1000)
            mid_thigh_acc_ml.append(float(line[6]) * physical_const / 1000)
            lowerback_acc_ap.append(float(line[7]) * physical_const / 1000)
            lowerback_acc_v.append(float(line[8]) * physical_const / 1000)
            lowerback_acc_ml.append(float(line[9]) * physical_const / 1000)
            gt.append(int(line[10]))    
    
    if len(ankle_acc_ap) >= 700:
        count += 1
        event = ['stand or walk or turn'] * len(ankle_acc_ap)
        all_data[f'rectified_{count}_daphnet'] = {
            'ori_filename':txt_file,
            'event': np.array(event), 
            'L_Ankle_Acc_AP': sample_normalize(torch.tensor(ankle_acc_ap, dtype=torch.float32)),
            'L_Ankle_Acc_V': sample_normalize(torch.tensor(ankle_acc_v, dtype=torch.float32)),
            'L_Ankle_Acc_ML': sample_normalize(torch.tensor(ankle_acc_ml, dtype=torch.float32)),
            'L_MidLatThigh_Acc_AP': sample_normalize(torch.tensor(mid_thigh_acc_ap, dtype=torch.float32)),
            'L_MidLatThigh_Acc_V': sample_normalize(torch.tensor(mid_thigh_acc_v, dtype=torch.float32)),
            'L_MidLatThigh_Acc_ML': sample_normalize(torch.tensor(mid_thigh_acc_ml, dtype=torch.float32)),
            'LowerBack_Acc_AP': sample_normalize(torch.tensor(lowerback_acc_ap, dtype=torch.float32)),
            'LowerBack_Acc_V': sample_normalize(torch.tensor(lowerback_acc_v, dtype=torch.float32)),
            'LowerBack_Acc_ML': sample_normalize(torch.tensor(lowerback_acc_ml, dtype=torch.float32)),
            'gt': torch.tensor(gt, dtype=torch.float32),
        }
    file.close()

joblib.dump(all_data, open(os.path.join('data/rectified_data/all_data', 
                                                f"all_daphnet.p"), 'wb'))


100%|██████████| 17/17 [00:05<00:00,  3.36it/s]


# kaggle_pd_data

In [2]:
lab_home = 'tdcsfog'
dpath = f'data/kaggle_pd_data/train/{lab_home}'
all_tdcsfog = [file for file in os.listdir(dpath) if file.endswith('csv')]

all_data = {}
count = 0
physical_const = physical_constants['standard acceleration of gravity'][0]

for csv_file in tqdm(all_tdcsfog, total=len(all_tdcsfog)):
    count += 1

    df = pd.read_csv(os.path.join(dpath, csv_file))
    df['gt'] = df.apply(lambda row: 1 if row[['StartHesitation', 'Turn', 'Walking']].any() \
                                            else 0, axis=1)
    df['event'] = df.apply(lambda row: 'start hesitation' if row['StartHesitation'] == 1 else
                                        'turn' if row['Turn'] == 1 else
                                        'walk' if row['Walking'] == 1 else
                                        'unlabeled', axis=1)

    all_data[f'rectified_{count}_kaggle_pd_data'] = {
        'ori_filename': f"{lab_home}_" + csv_file,
        'event': df['event'].to_numpy(), 
        'LowerBack_Acc_V': sample_normalize(torch.tensor(df['AccV'].to_numpy(), 
                                                         dtype=torch.float32)),
        'LowerBack_Acc_ML': sample_normalize(torch.tensor(df['AccML'].to_numpy(), 
                                                          dtype=torch.float32)),
        'LowerBack_Acc_AP': sample_normalize(torch.tensor(df['AccAP'].to_numpy(), 
                                                          dtype=torch.float32)),
        'gt': torch.tensor(df['gt'].to_numpy(), dtype=torch.float32),
    }
        
    # print(all_data[f'rectified_{count}_kaggle_pd_data'].keys())
    # print(all_data[f'rectified_{count}_kaggle_pd_data']['ori_filename'])
    # print(all_data[f'rectified_{count}_kaggle_pd_data']['event'].shape)
    # print(all_data[f'rectified_{count}_kaggle_pd_data']['LowerBack_Acc_V'].shape)
    # print(all_data[f'rectified_{count}_kaggle_pd_data']['LowerBack_Acc_V'].dtype)
    # print(all_data[f'rectified_{count}_kaggle_pd_data']['gt'].shape)
    # print(all_data[f'rectified_{count}_kaggle_pd_data']['gt'].dtype)
joblib.dump(all_data, open(os.path.join('data/rectified_data/all_data', 
                                        f"all_kaggle_pd_data_{lab_home}.p"), 'wb'))
    

100%|██████████| 833/833 [24:44<00:00,  1.78s/it] 


In [2]:
TASK = {
    '4MW': '4-meter walk',
    'TUG-ST': 'stand up, walk, return and sit down',
    'TUG-DT': 'stand up, walk, return and sit down while subtracting numbers',
    'Turning-ST': 'performing 360-degree turns 4 times, each time alternating the rotation direction',
    'Turning-DT': 'performing 360-degree turns 4 times, each time alternating the rotation direction, and subtracting numbers at the same time',
    'Hotspot1': 'a walking trial that involves opening a door, entering another room, turning, and returning to the start point',
    'Hotspot2': 'walking through an area in the house',
    'MB1': 'sit to stand', 
    'MB2a': 'rise to toes',
    'MB2b': 'rise to toes',
    'MB3-L': 'stand on the left leg',
    'MB3-R': 'stand on the right leg',
    'MB4': 'compensatory stepping correction - forward',
    'MB5': 'compensatory stepping correction - backward',
    'MB6-L': 'compensatory stepping correction - left lateral',
    'MB6-R': 'compensatory stepping correction - right lateral',
    'MB7': 'stand in the posture: feet together, eyes open, and firm surface',
    'MB8': 'stand in the posture: feet together, eyes closed, and foam surface',
    'MB9': 'incline eyes closed (shoulder width, arms at your side)',
    'MB10': 'change in gait speed',
    'MB11': 'walk with head turns - horizontal',
    'MB12': 'walk with pivot turns',
    'MB13': 'step over obstacles',
}

lab_home = 'defog'
dpath = f'data/kaggle_pd_data/train/{lab_home}'
all_tdcsfog = [file for file in os.listdir(dpath) if file.endswith('csv')]

all_data = {}
count = 0
physical_const = physical_constants['standard acceleration of gravity'][0]

tasks_txt_file = open('data/kaggle_pd_data/tasks.csv')
all_lines = tasks_txt_file.readlines()

count = 0
for line in tqdm(all_lines, total=len(all_lines)):
    line = line.strip().split(',')
    if line[-1] not in TASK.keys():
        continue
    csv_file = line[0] + '.csv'

    try:
        df = pd.read_csv(os.path.join(dpath, csv_file))
    except:
        continue

    start_t = int(math.floor(float(line[1]) * 100))
    end_t = int(math.ceil(float(line[2]) * 100))

    while not df['Valid'].iloc[start_t] or not df['Task'].iloc[start_t]:
        start_t += 1
        
    while not df['Valid'].iloc[end_t] or not df['Task'].iloc[end_t]:
        end_t -= 1
        
    if (end_t - start_t + 1) < 900:
        continue

    df = df.iloc[start_t:end_t+1]
    
    df['gt'] = df.apply(lambda row: 1 if row[['StartHesitation', 'Turn', 'Walking']].any() \
                                            else 0, axis=1)

    event = [TASK[line[-1]]] * (end_t - start_t + 1) 
    event = np.array(event)
    
    count += 1
    
    
    # print(df['AccV'].iloc[:3])

    all_data[f'rectified_{count}_kaggle_pd_data'] = {
        'ori_filename': f"{lab_home}_" + csv_file,
        'event': event, 
        'LowerBack_Acc_V': sample_normalize(torch.tensor(df['AccV'].to_numpy(), 
                                                         dtype=torch.float32) * physical_const),
        'LowerBack_Acc_ML': sample_normalize(torch.tensor(df['AccML'].to_numpy(), 
                                                          dtype=torch.float32) * physical_const),
        'LowerBack_Acc_AP': sample_normalize(torch.tensor(df['AccAP'].to_numpy(), 
                                                          dtype=torch.float32) * physical_const),
        'gt': torch.tensor(df['gt'].to_numpy(), dtype=torch.float32),
    }
    
    # print(all_data[f'rectified_{count}_kaggle_pd_data'].keys())
    # print(all_data[f'rectified_{count}_kaggle_pd_data']['ori_filename'])
    # print(all_data[f'rectified_{count}_kaggle_pd_data']['event'].shape)
    # print(all_data[f'rectified_{count}_kaggle_pd_data']['LowerBack_Acc_V'][:3])
    # break
    # print(all_data[f'rectified_{count}_kaggle_pd_data']['gt'].shape)
    # print(all_data[f'rectified_{count}_kaggle_pd_data']['gt'].dtype)
joblib.dump(all_data, open(os.path.join('data/rectified_data/all_data', 
                                        f"all_kaggle_pd_data_{lab_home}.p"), 'wb'))
print('num trials: ', len(all_data.keys()))
    

100%|██████████| 2818/2818 [13:18<00:00,  3.53it/s]  


num trials:  1027


# turn_in_place_l

In [5]:

pdfeinfo = pd.read_csv('data/turn_in_place/pdfeinfo.csv')
dpath = 'data/turn_in_place/IMU'
all_txt_files = [file for file in os.listdir(dpath) \
                if file.endswith('txt') and not file.endswith('standing.txt')]

all_data = {}
count = 0
physical_const = physical_constants['standard acceleration of gravity'][0]
for txt_file in tqdm(all_txt_files, total=len(all_txt_files)):
    count += 1
    subject_id = txt_file[3:5]
    session_num = txt_file[6]
    row = pdfeinfo[pdfeinfo['id'] == f'pdfe{subject_id}']
    l_or_r = 'L' if row.iloc[0]['more_affected_side'] == 'left' else 'R'

    if l_or_r == 'R':
        continue

    tug_time = row.iloc[0][f's{session_num}_tug_sec']
    tug_dual_time = row.iloc[0][f's{session_num}_tug_dual_sec']

    frame_idx = []
    event = []
    acc_ml, acc_ap, acc_si = [], [], []
    gyr_ml, gyr_ap, gyr_si = [], [], []
    gt = []
    
    with open(os.path.join(dpath, txt_file), 'r') as file:
        all_lines = file.readlines()
        
        for line in all_lines[1:]:
            values = line.strip().split('\t')
            annotation = 'turn in place' if float(values[1]) >= tug_time else 'unlabeled'
            annotation = 'turn in place with additional tasks' \
                                if float(values[1]) >= tug_dual_time else annotation

            frame_idx.append(int(values[0]))
            event.append(annotation)
            # g --> m/s^2
            acc_ml.append(float(values[2]) * physical_const)
            acc_ap.append(float(values[3]) * physical_const)
            acc_si.append(float(values[4]) * physical_const)
            gyr_ml.append(float(values[5]) * degree)
            gyr_ap.append(float(values[6]) * degree)
            gyr_si.append(float(values[7]) * degree)
            
            gt.append(int(values[8]))
            
            # print(acc_x[-1], acc_z[-1], acc_y[-1])
            # print(gyr_x[-1], gyr_z[-1], gyr_y[-1])
            # break
        
        # print(torch.tensor(acc_x)[:3])
        all_data[f'rectified_{count}_turn_in_place_l'] = {
            'ori_filename':txt_file,
            'frame_idx': torch.tensor(frame_idx), 
            'event': np.array(event), 
            f'{l_or_r}_LatShank_Acc_ML': sample_normalize(torch.tensor(acc_ml, dtype=torch.float32)),
            f'{l_or_r}_LatShank_Acc_AP': sample_normalize(torch.tensor(acc_ap, dtype=torch.float32)),
            f'{l_or_r}_LatShank_Acc_SI': sample_normalize(torch.tensor(acc_si, dtype=torch.float32)),
            f'{l_or_r}_LatShank_Gyr_ML': sample_normalize(torch.tensor(gyr_ml, dtype=torch.float32)),
            f'{l_or_r}_LatShank_Gyr_AP': sample_normalize(torch.tensor(gyr_ap, dtype=torch.float32)),
            f'{l_or_r}_LatShank_Gyr_SI': sample_normalize(torch.tensor(gyr_si, dtype=torch.float32)),
            'gt': torch.tensor(gt, dtype=torch.float32),
        }
        
        # print(all_data[f'rectified_{count}_turn_in_place'].keys())
        # print(all_data[f'rectified_{count}_turn_in_place']['ori_filename'])
        # print(all_data[f'rectified_{count}_turn_in_place']['event'].shape)
        # print(all_data[f'rectified_{count}_turn_in_place'][f'{l_or_r}_LatShank_Acc_X'].shape)
        # print(all_data[f'rectified_{count}_turn_in_place'][f'{l_or_r}_LatShank_Acc_X'].dtype)
    joblib.dump(all_data, open(os.path.join('data/rectified_data/all_data', 
                                                f"all_turn_in_place_l.p"), 'wb'))
    

  0%|          | 0/71 [00:00<?, ?it/s]

100%|██████████| 71/71 [00:05<00:00, 13.39it/s]


# turn_in_place_r

In [6]:

pdfeinfo = pd.read_csv('data/turn_in_place/pdfeinfo.csv')
dpath = 'data/turn_in_place/IMU'
all_txt_files = [file for file in os.listdir(dpath) \
                if file.endswith('txt') and not file.endswith('standing.txt')]

all_data = {}
count = 0
physical_const = physical_constants['standard acceleration of gravity'][0]
for txt_file in tqdm(all_txt_files, total=len(all_txt_files)):
    count += 1
    subject_id = txt_file[3:5]
    session_num = txt_file[6]
    row = pdfeinfo[pdfeinfo['id'] == f'pdfe{subject_id}']
    l_or_r = 'L' if row.iloc[0]['more_affected_side'] == 'left' else 'R'

    if l_or_r == 'L':
        continue

    tug_time = row.iloc[0][f's{session_num}_tug_sec']
    tug_dual_time = row.iloc[0][f's{session_num}_tug_dual_sec']

    frame_idx = []
    event = []
    acc_ml, acc_ap, acc_si = [], [], []
    gyr_ml, gyr_ap, gyr_si = [], [], []
    gt = []
    
    with open(os.path.join(dpath, txt_file), 'r') as file:
        all_lines = file.readlines()
        
        for line in all_lines[1:]:
            values = line.strip().split('\t')
            annotation = 'turn in place' if float(values[1]) >= tug_time else 'unlabeled'
            annotation = 'turn in place with additional tasks' \
                                if float(values[1]) >= tug_dual_time else annotation

            frame_idx.append(int(values[0]))
            event.append(annotation)
            # g --> m/s^2
            acc_ml.append(float(values[2]) * physical_const)
            acc_ap.append(float(values[3]) * physical_const)
            acc_si.append(float(values[4]) * physical_const)
            gyr_ml.append(float(values[5]) * degree)
            gyr_ap.append(float(values[6]) * degree)
            gyr_si.append(float(values[7]) * degree)
            
            gt.append(int(values[8]))
            
            # print(acc_x[-1], acc_z[-1], acc_y[-1])
            # print(gyr_x[-1], gyr_z[-1], gyr_y[-1])
            # break
        
        # print(torch.tensor(acc_x)[:3])
        all_data[f'rectified_{count}_turn_in_place_r'] = {
            'ori_filename':txt_file,
            'frame_idx': torch.tensor(frame_idx), 
            'event': np.array(event), 
            f'{l_or_r}_LatShank_Acc_ML': sample_normalize(torch.tensor(acc_ml, dtype=torch.float32)),
            f'{l_or_r}_LatShank_Acc_AP': sample_normalize(torch.tensor(acc_ap, dtype=torch.float32)),
            f'{l_or_r}_LatShank_Acc_SI': sample_normalize(torch.tensor(acc_si, dtype=torch.float32)),
            f'{l_or_r}_LatShank_Gyr_ML': sample_normalize(torch.tensor(gyr_ml, dtype=torch.float32)),
            f'{l_or_r}_LatShank_Gyr_AP': sample_normalize(torch.tensor(gyr_ap, dtype=torch.float32)),
            f'{l_or_r}_LatShank_Gyr_SI': sample_normalize(torch.tensor(gyr_si, dtype=torch.float32)),
            'gt': torch.tensor(gt, dtype=torch.float32),
        }
        
        # print(all_data[f'rectified_{count}_turn_in_place'].keys())
        # print(all_data[f'rectified_{count}_turn_in_place']['ori_filename'])
        # print(all_data[f'rectified_{count}_turn_in_place']['event'].shape)
        # print(all_data[f'rectified_{count}_turn_in_place'][f'{l_or_r}_LatShank_Acc_X'].shape)
        # print(all_data[f'rectified_{count}_turn_in_place'][f'{l_or_r}_LatShank_Acc_X'].dtype)
    joblib.dump(all_data, open(os.path.join('data/rectified_data/all_data', 
                                                f"all_turn_in_place_r.p"), 'wb'))
    

100%|██████████| 71/71 [00:05<00:00, 13.04it/s]
